In [ ]:
#!/usr/bin/env python3
# ΔSWE map with 0.5 mm bins, integer-only colorbar ticks, and narrow grey zero bin (±0.001)
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import BoundaryNorm, ListedColormap
from matplotlib.cm import get_cmap

# Fortran-snippet parameters
alpha    = 0.60   # SCF_ANA_ALPHA
beta     = 0.55   # SCF_ANA_BETA
incr_max = 5.0    # SCF_ANA_MAXINCRSWE [mm]

# Grid
scf_obs_pct = np.linspace(0.0, 100.0, 801)
scf_mod_pct = np.linspace(0.0, 100.0, 801)
X_obs, Y_mod = np.meshgrid(scf_obs_pct, scf_mod_pct)
x = X_obs / 100.0
y = Y_mod / 100.0

# ΔSWE rule
delta_swe = np.zeros_like(x)
add_mask = y < alpha * x
delta_swe[add_mask] = incr_max * (x[add_mask] - y[add_mask] / alpha)
remove_mask = (~add_mask) & (x < beta)
delta_swe[remove_mask] = -incr_max * y[remove_mask] * (1.0 - x[remove_mask] / beta)
delta_swe = np.clip(delta_swe, -incr_max, incr_max)

# 0.5 mm bin edges, with special narrow zero bin
levels = list(np.arange(-5.0, -0.5, 0.5)) + [-0.5, -0.001, 0.001, 0.5] + list(np.arange(1.0, 5.0 + 0.5, 0.5))

# Discrete warm–cool colormap with grey center bin
n_bins = len(levels) - 1
base = get_cmap("coolwarm", n_bins)
colors = base(np.arange(n_bins))
zero_bin_index = levels.index(-0.001)   # bin (-0.001, 0.001)
colors[zero_bin_index] = [0.7, 0.7, 0.7, 1.0]  # grey
cmap = ListedColormap(colors)
norm = BoundaryNorm(levels, ncolors=cmap.N, clip=True)

# Plot
fig, ax = plt.subplots(figsize=(8, 6))
im = ax.imshow(
    delta_swe,
    origin="lower",
    extent=[scf_obs_pct.min(), scf_obs_pct.max(), scf_mod_pct.min(), scf_mod_pct.max()],
    aspect="auto", interpolation="nearest", cmap=cmap, norm=norm
)

# Colorbar: show only integer tick labels
ticks = np.arange(-5, 6, 1)
cbar = plt.colorbar(im, ax=ax, boundaries=levels, ticks=ticks, spacing="proportional")
cbar.set_label("SWE Increment [mm]")

# Decision boundaries
x_line = np.array([0.0, 100.0])
y_line = alpha * (x_line / 100.0) * 100.0
ax.plot([beta * 100.0, beta * 100.0], [0.0, 100.0], color="0.15", linewidth=3)
ax.plot(x_line, y_line, color="0.15", linewidth=3)

ax.set_xlabel("MODIS SCF Obs [%]")
ax.set_ylabel("Model SCF [%]")
ax.set_xlim(0, 100); ax.set_ylim(0, 100)
ax.set_title(f"ΔSWE Map (α={alpha:.2f}, β={beta:.2f}, incr_max={incr_max:.1f} mm)")
plt.tight_layout()
plt.savefig("swe_increment_map_bins05_ticks1_grey0.png", dpi=200, bbox_inches="tight")
plt.show()
